# The Vanishing/Exploding Gradients Problems
 * The vanishing gradients problem occurs when the gradients of the loss function with respect to the parameters become extremely small during backpropagation.
 * Conversely, the exploding gradients problem arises when the gradients of the loss function grow exponentially as they propagate backward through the layers during backpropagation.


## Weight Initialization
 * Proper initialization of network weights can help alleviate both vanishing and exploding gradients problems. 
 * Techniques like __Xavier__ initialization and __He__ initialization are commonly used to ensure that the weights are initialized in a way that keeps the signal propagated through the network.

In [1]:
# Importing essential package
import numpy as np
import tensorflow as tf
from tensorflow import keras

# List comprehension to get the names of initializers in Keras without underscores
initializer_names = [name for name in dir(keras.initializers) if not name.startswith("_")]

# Printing the initializer names
print(initializer_names)

['Constant', 'GlorotNormal', 'GlorotUniform', 'HeNormal', 'HeUniform', 'Identity', 'Initializer', 'LecunNormal', 'LecunUniform', 'Ones', 'Orthogonal', 'RandomNormal', 'RandomUniform', 'TruncatedNormal', 'VarianceScaling', 'Zeros', 'constant', 'deserialize', 'get', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform', 'identity', 'lecun_normal', 'lecun_uniform', 'ones', 'orthogonal', 'random_normal', 'random_uniform', 'serialize', 'truncated_normal', 'variance_scaling', 'zeros']


* Creating a dense layer with built-in initializers

In [2]:
# Creating a dense layer with 10 units, ReLU activation, and He normal weight initialization
keras.layers.Dense(10, activation="relu", kernel_initializer="he_normal")

* Defining an initializer with Variance Scaling (custom kernel initializer)

In [3]:
# Defining an initializer with Variance Scaling
init = keras.initializers.VarianceScaling(scale=2., mode='fan_avg', distribution='uniform')

# Creating a dense layer with 10 units, ReLU activation, and custom kernel initializer
dense_layer = keras.layers.Dense(10, activation="relu", kernel_initializer=init)

## Nonsaturating Activation Functions
 * Non-saturating activation functions refer to activation functions that do not suffer from the vanishing gradient problem to the same extent as traditional saturating activation functions like sigmoid and tanh. These functions typically have gradients that do not diminish as quickly as the __sigmoid__ or __tanh__ functions for large input values, thereby alleviating the vanishing gradient problem. 
 * One prominent example of a non-saturating activation function is the Rectified Linear Unit (__ReLU__) and its variants.

1. __Rectified Linear Unit (ReLU):__
    * ReLU is a popular activation function in deep learning, defined as __f(x)=max(0,x)__. It outputs zero for negative inputs and passes positive inputs unchanged. 
    * It's computationally efficient and doesn't suffer from vanishing gradients for positive inputs. However, it may lead to the __"dying ReLU"__ problem, causing neurons to become inactive for negative inputs. 
    * Variants like __Leaky ReLU__, __Parametric ReLU (PReLU)__, and __Exponential Linear Unit (ELU)__ address this issue by allowing a small, non-zero gradient for negative inputs.

In [4]:
# Listing ReLU variants in Keras.layers module
[m for m in dir(keras.layers) if "relu" in m.lower()]

['LeakyReLU', 'PReLU', 'ReLU', 'ThresholdedReLU']

2. __Leaky ReLU:__
    * Leaky ReLU is an activation function similar to ReLU but with a small, non-zero gradient for negative input values. 
    * It's defined as __f(x)=max(αx,x)__, where α is a small constant (e.g., 0.01). 
    * Leaky ReLU addresses the __"dying ReLU"__ problem by preventing neurons from becoming inactive for negative inputs during training. It retains the efficiency of ReLU while mitigating its limitations.

In [5]:
# Training a neural network on Fashion MNIST using the Leaky ReLU:

# Loading Fashion MNIST dataset and preprocessing
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

# Setting random seeds for reproducibility
tf.random.set_seed(42)
np.random.seed(42)

# Defining the neural network model with Leaky ReLU activation
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, kernel_initializer="he_normal"),
    keras.layers.LeakyReLU(),  # Leaky ReLU activation
    keras.layers.Dense(100, kernel_initializer="he_normal"),
    keras.layers.LeakyReLU(),  # Leaky ReLU activation
    keras.layers.Dense(10, activation="softmax")
])

# Compiling the model
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])

# Training the model
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid))

Epoch 1/10
1719/1719 [==============================] - 3s 1ms/step - loss: 1.3039 - accuracy: 0.6001 - val_loss: 0.8886 - val_accuracy: 0.7040
Epoch 2/10
1719/1719 [==============================] - 2s 1ms/step - loss: 0.8033 - accuracy: 0.7293 - val_loss: 0.7169 - val_accuracy: 0.7654
Epoch 3/10
1719/1719 [==============================] - 2s 1ms/step - loss: 0.6888 - accuracy: 0.7704 - val_loss: 0.6480 - val_accuracy: 0.7912
Epoch 4/10
1719/1719 [==============================] - 2s 1ms/step - loss: 0.6282 - accuracy: 0.7897 - val_loss: 0.5948 - val_accuracy: 0.8050
Epoch 5/10
1719/1719 [==============================] - 2s 1ms/step - loss: 0.5891 - accuracy: 0.8029 - val_loss: 0.5629 - val_accuracy: 0.8162
Epoch 6/10
1719/1719 [==============================] - 2s 1ms/step - loss: 0.5610 - accuracy: 0.8107 - val_loss: 0.5400 - val_accuracy: 0.8208
Epoch 7/10
1719/1719 [==============================] - 2s 1ms/step - loss: 0.5396 - accuracy: 0.8165 - val_loss: 0.5197 - val_accuracy:

3. __Parametric ReLU:__
    * Parametric ReLU (PReLU) extends Leaky ReLU by allowing the coefficient α to be learned during training rather than fixed. PReLU can adaptively adjust the leakage for negative inputs based on data, potentially improving performance over fixed coefficients.

In [6]:
# Training a neural network on Fashion MNIST using the Leaky PReLU:

# Loading Fashion MNIST dataset and preprocessing
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

# Setting random seeds for reproducibility
tf.random.set_seed(42)
np.random.seed(42)

# Defining the neural network model with Leaky ReLU activation
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, kernel_initializer="he_normal"),
    keras.layers.PReLU(),
    keras.layers.Dense(100, kernel_initializer="he_normal"),
    keras.layers.PReLU(),
    keras.layers.Dense(10, activation="softmax")
])

# Compiling the model
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])

# Training the model
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid))

Epoch 1/10
1719/1719 [==============================] - 3s 2ms/step - loss: 1.3511 - accuracy: 0.6177 - val_loss: 0.9070 - val_accuracy: 0.7164
Epoch 2/10
1719/1719 [==============================] - 2s 1ms/step - loss: 0.8104 - accuracy: 0.7421 - val_loss: 0.7203 - val_accuracy: 0.7658
Epoch 3/10
1719/1719 [==============================] - 2s 1ms/step - loss: 0.6907 - accuracy: 0.7753 - val_loss: 0.6477 - val_accuracy: 0.7902
Epoch 4/10
1719/1719 [==============================] - 2s 1ms/step - loss: 0.6274 - accuracy: 0.7947 - val_loss: 0.5915 - val_accuracy: 0.8088
Epoch 5/10
1719/1719 [==============================] - 2s 1ms/step - loss: 0.5860 - accuracy: 0.8073 - val_loss: 0.5570 - val_accuracy: 0.8178
Epoch 6/10
1719/1719 [==============================] - 2s 1ms/step - loss: 0.5560 - accuracy: 0.8147 - val_loss: 0.5329 - val_accuracy: 0.8224
Epoch 7/10
1719/1719 [==============================] - 2s 1ms/step - loss: 0.5334 - accuracy: 0.8215 - val_loss: 0.5128 - val_accuracy:

4. __Exponential Linear Unit (ELU):__
    * ELU is defined as f(x)=x if x≥0 and f(x)=α(e^x−1) if x<0, where α is a small positive constant.
    * ELU behaves similarly to ReLU for positive input values but has a smooth curve for negative input values, allowing it to handle negative inputs more gracefully than ReLU. 
    * ELU also prevents the "dying ReLU" problem by ensuring a non-zero gradient for all inputs.

In [7]:
# Training a neural network on Fashion MNIST using the Leaky PReLU:

# Loading Fashion MNIST dataset and preprocessing
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

# Setting random seeds for reproducibility
tf.random.set_seed(42)
np.random.seed(42)

# Defining the neural network model with Leaky ReLU activation
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, kernel_initializer="he_normal", activation="elu"),
    keras.layers.Dense(100, kernel_initializer="he_normal", activation="elu"),
    keras.layers.Dense(10, activation="softmax")
])

# Compiling the model
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])

# Training the model
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid))

Epoch 1/10
1719/1719 [==============================] - 3s 1ms/step - loss: 1.1413 - accuracy: 0.6345 - val_loss: 0.7835 - val_accuracy: 0.7384
Epoch 2/10
1719/1719 [==============================] - 2s 1ms/step - loss: 0.7218 - accuracy: 0.7588 - val_loss: 0.6498 - val_accuracy: 0.7880
Epoch 3/10
1719/1719 [==============================] - 2s 1ms/step - loss: 0.6317 - accuracy: 0.7884 - val_loss: 0.5951 - val_accuracy: 0.8026
Epoch 4/10
1719/1719 [==============================] - 2s 1ms/step - loss: 0.5835 - accuracy: 0.8028 - val_loss: 0.5515 - val_accuracy: 0.8178
Epoch 5/10
1719/1719 [==============================] - 2s 1ms/step - loss: 0.5524 - accuracy: 0.8100 - val_loss: 0.5262 - val_accuracy: 0.8274
Epoch 6/10
1719/1719 [==============================] - 2s 1ms/step - loss: 0.5301 - accuracy: 0.8175 - val_loss: 0.5073 - val_accuracy: 0.8296
Epoch 7/10
1719/1719 [==============================] - 2s 1ms/step - loss: 0.5130 - accuracy: 0.8233 - val_loss: 0.4923 - val_accuracy:

5. __Scaled Exponential Linear Unit (SELU):__
    * It's an activation function introduced to address the vanishing/exploding gradient problem while promoting self-normalizing properties in neural networks.
    * SELU has a self-normalizing property, meaning the output distribution remains close to mean 0 and standard deviation 1, which helps stabilize training.
    * It overcomes the vanishing/exploding gradient problem by promoting stable mean and variance propagation through the network.
    * It has an exponential component for negative inputs, ensuring non-zero gradients, thus avoiding the "dying ReLU" problem.
    * SELU is specifically designed for feedforward neural networks, and its performance may vary depending on the architecture and hyperparameters.

In [8]:
# Training a neural network on Fashion MNIST using the Leaky SeLU:

# Loading Fashion MNIST dataset and preprocessing
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
# Scaling pixel values to the range [0, 1]
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
# Splitting training set into validation and training subsets
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

# Calculating mean and standard deviation of training pixel values
pixel_means = X_train.mean(axis=0, keepdims=True)
pixel_stds = X_train.std(axis=0, keepdims=True)
# Standardizing the input features using mean and standard deviation
X_train_scaled = (X_train - pixel_means) / pixel_stds
X_valid_scaled = (X_valid - pixel_means) / pixel_stds
X_test_scaled = (X_test - pixel_means) / pixel_stds

# Setting random seeds for reproducibility
tf.random.set_seed(42)
np.random.seed(42)

# Constructing the neural network model with SELU activation
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[28, 28]))
model.add(keras.layers.Dense(300, activation="selu", kernel_initializer="lecun_normal"))
# Adding 99 hidden layers with SELU activation
for layer in range(99):
    model.add(keras.layers.Dense(100, activation="selu", kernel_initializer="lecun_normal"))
# Adding output layer with softmax activation for 10 classes
model.add(keras.layers.Dense(10, activation="softmax"))

# Compiling the model with sparse categorical crossentropy loss and SGD optimizer
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])

# Training the model on scaled training data for 5 epochs with validation data for monitoring
history = model.fit(X_train_scaled, y_train, epochs=5,
                    validation_data=(X_valid_scaled, y_valid))

Epoch 1/5
1719/1719 [==============================] - 35s 19ms/step - loss: 1.2287 - accuracy: 0.5356 - val_loss: 1.0872 - val_accuracy: 0.6288
Epoch 2/5
1719/1719 [==============================] - 32s 19ms/step - loss: 0.8088 - accuracy: 0.7109 - val_loss: 0.6483 - val_accuracy: 0.7766
Epoch 3/5
1719/1719 [==============================] - 32s 18ms/step - loss: 0.6579 - accuracy: 0.7627 - val_loss: 0.6145 - val_accuracy: 0.7766
Epoch 4/5
1719/1719 [==============================] - 32s 19ms/step - loss: 0.5760 - accuracy: 0.7930 - val_loss: 0.5408 - val_accuracy: 0.8044
Epoch 5/5
1719/1719 [==============================] - 32s 18ms/step - loss: 0.5315 - accuracy: 0.8124 - val_loss: 0.5244 - val_accuracy: 0.8100


## Batch Normalization
 * Batch Normalization is a technique used in deep learning to enhance the training process of neural networks. 
 * This means that it adjusts the activations of each layer so that they have a mean of zero and a standard deviation of one. By doing this normalization, Batch Normalization helps to stabilize the training process and improve the speed at which neural networks converge to an optimal solution.

 * Batch Normalization has become a standard technique in deep learning because it contributes to __faster training__, __improved model performance__, and __greater stability__ during the training process.

In [12]:
# Training a neural network on Fashion MNIST using the Batch Normalization:

# Loading Fashion MNIST dataset and preprocessing
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
# Scaling pixel values to the range [0, 1]
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
# Splitting training set into validation and training subsets
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]


# Setting random seeds for reproducibility
tf.random.set_seed(42)
np.random.seed(42)

# Define a Sequential model
model = keras.models.Sequential([
    # Flatten layer to convert 2D input into 1D
    keras.layers.Flatten(input_shape=[28, 28]),
    # BatchNormalization layer applied after the Flatten layer
    keras.layers.BatchNormalization(),
    # Dense layer with 300 neurons and ReLU activation function
    keras.layers.Dense(300, activation="relu"),
    # BatchNormalization layer applied after the first Dense layer
    keras.layers.BatchNormalization(),
    # Dense layer with 100 neurons and ReLU activation function
    keras.layers.Dense(100, activation="relu"),
    # BatchNormalization layer applied after the second Dense layer
    keras.layers.BatchNormalization(),
    # Dense layer with 10 neurons and softmax activation function for classification
    keras.layers.Dense(10, activation="softmax")
])

# Compiling the model with sparse categorical crossentropy loss and SGD optimizer
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])

# Training the model on training data for 10 epochs with validation data for monitoring
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid))

Epoch 1/10
1719/1719 [==============================] - 4s 2ms/step - loss: 0.8559 - accuracy: 0.7162 - val_loss: 0.5683 - val_accuracy: 0.8082
Epoch 2/10
1719/1719 [==============================] - 3s 2ms/step - loss: 0.5742 - accuracy: 0.8023 - val_loss: 0.4850 - val_accuracy: 0.8386
Epoch 3/10
1719/1719 [==============================] - 3s 2ms/step - loss: 0.5207 - accuracy: 0.8193 - val_loss: 0.4469 - val_accuracy: 0.8472
Epoch 4/10
1719/1719 [==============================] - 3s 2ms/step - loss: 0.4808 - accuracy: 0.8339 - val_loss: 0.4245 - val_accuracy: 0.8554
Epoch 5/10
1719/1719 [==============================] - 3s 2ms/step - loss: 0.4536 - accuracy: 0.8404 - val_loss: 0.4088 - val_accuracy: 0.8610
Epoch 6/10
1719/1719 [==============================] - 3s 2ms/step - loss: 0.4367 - accuracy: 0.8462 - val_loss: 0.3978 - val_accuracy: 0.8638
Epoch 7/10
1719/1719 [==============================] - 3s 2ms/step - loss: 0.4247 - accuracy: 0.8503 - val_loss: 0.3853 - val_accuracy:

In [10]:
# Display a summary of the model architecture
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_4 (Flatten)         (None, 784)               0         
                                                                 
 batch_normalization (BatchN  (None, 784)              3136      
 ormalization)                                                   
                                                                 
 dense_112 (Dense)           (None, 300)               235500    
                                                                 
 batch_normalization_1 (Batc  (None, 300)              1200      
 hNormalization)                                                 
                                                                 
 dense_113 (Dense)           (None, 100)               30100     
                                                                 
 batch_normalization_2 (Batc  (None, 100)             

In [11]:
# Retrieve the BatchNormalization layer after the first layer in the model
bn1 = model.layers[1]

# List the names and trainability of the variables in the BatchNormalization layer
# This can be useful for inspecting the variables of the BatchNormalization layer
[(var.name, var.trainable) for var in bn1.variables]


[('batch_normalization/gamma:0', True),
 ('batch_normalization/beta:0', True),
 ('batch_normalization/moving_mean:0', False),
 ('batch_normalization/moving_variance:0', False)]